In [67]:
from image_acquisition.image_sensor import *
from image_acquisition.playback_images import *
from image_acquisition.video_file_writer import *
#from image_acquisition.ueye_python_driver import UEyePythonDriver

from feature_computation.object_tracking import *
from feature_computation.object_detection_and_tracking import *
from feature_computation.visualization import *

import matplotlib as plt
import csv
from datetime import datetime

In [68]:
# Load the ONNX model

output_path = "../../../makerspace_20241126/output/object_tracking_20241127.mp4"
raw_output_path =  "../../../makerspace_20241126/output/object_tracking_raw_20241127.mp4"


onnx_model_path = "../../makerspace_20241126/export/onnx/simplified-model.onnx"
print(os.path.realpath(onnx_model_path))
video_path = "../../makerspace_20241126/videos/soldering_recordings/Stefan/rec_1.avi"
print(os.path.realpath(video_path))

trackers = {
   "pad": MultiObjectTracker(class_ids=[0,1,4,5], disappearance_threshold=-1, ratio_threshold=0.7),
   "cabel": MultiObjectTracker(class_ids=2, disappearance_threshold=50, ratio_threshold=0.7),
   "soldering_iron": KalmanFilterTracker(class_ids=3, ratio_threshold=0.7, disappearance_threshold=70)
}

C:\Users\ricar\Documents\Programming\makerspace_20241126\export\onnx\simplified-model.onnx
C:\Users\ricar\Documents\Programming\makerspace_20241126\videos\soldering_recordings\Stefan\rec_1.avi


In [69]:

visualizer = ObjectTrackerVisualization(trackers)

In [70]:
import state_update as su

pads = [su.pad(), su.pad()]

In [71]:
def float_state_to_string_state(float_state):
    if float_state == 0.0:
        return "presoldered"
    if float_state == 1.0:
        return "soldered"
    if float_state == 4.0:
        return "unsoldered"
    if float_state == 5.0:
        return "in_process"
    raise Exception("unknown pad state")

csv_file_name = "output.csv"
csv_file = open(csv_file_name, mode='w', newline='')
fieldnames = [
    "Time", 
    "pad_0_state", "pad_0_x", "pad_0_y", "pad_0_w", "pad_0_h",
    "pad_1_state", "pad_1_x", "pad_1_y", "pad_1_w", "pad_1_h",
    "cabel_0_x", "cabel_0_y", "cabel_0_w", "cabel_0_h",
    "cabel_1_x", "cabel_1_y", "cabel_1_w", "cabel_1_h"
]
csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
csv_writer.writeheader() 

try:
    with VideoFileImageSensor(video_path=video_path) as img_src:
        with video_file_writer_matching_image_sensor(img_src, output_path) as writer:
            with onnx_network_and_trackers(onnx_model_path, trackers.values()) as network_and_trackers:

                def callback(frame: np.ndarray):
                    network_and_trackers.update_trackers(frame)
                    disp_img = frame.copy()
                    visualizer.draw_visualization_on_frame(disp_img)

                    height, width = disp_img.shape[:2]
                    new_width, new_height = width // 3, height // 3
                    disp_img = cv2.resize(disp_img, (new_width, new_height), interpolation=cv2.INTER_AREA)
                    cv2.imshow("object tracking", disp_img)
                    writer.write(disp_img)

                    current_time = datetime.now().strftime("%H:%M:%S")
                    csv_row = {"Time": current_time}

                    detected_pads = trackers["pad"].get_objects()
                    for i in range(2): 
                        if i < len(detected_pads):
                            dp = detected_pads[i]
                            pads[dp.id].pad_pos = dp.bounding_box
                            pads[dp.id].pad_state = float_state_to_string_state(dp.state)

                            csv_row[f"pad_{i}_state"] = pads[dp.id].pad_state
                            csv_row[f"pad_{i}_x"] = pads[dp.id].pad_pos.x.item() if pads[dp.id].pad_pos else None
                            csv_row[f"pad_{i}_y"] = pads[dp.id].pad_pos.y.item() if pads[dp.id].pad_pos else None
                            csv_row[f"pad_{i}_w"] = pads[dp.id].pad_pos.w.item() if pads[dp.id].pad_pos else None
                            csv_row[f"pad_{i}_h"] = pads[dp.id].pad_pos.h.item() if pads[dp.id].pad_pos else None

                            print(f"state of pad {i}: {pads[dp.id].pad_state}")
                            #print(f"state of pad {i}: {pads[dp.id].state}")
                        else:
                            csv_row[f"pad_{i}_state"] = "no pad detected"
                            csv_row[f"pad_{i}_x"] = None
                            csv_row[f"pad_{i}_y"] = None
                            csv_row[f"pad_{i}_w"] = None
                            csv_row[f"pad_{i}_h"] = None

                    su.wire_pos = [w.bounding_box for w in trackers["cabel"].get_objects()]
                    for i in range(2): 
                        if i < len(su.wire_pos):
                            wire = su.wire_pos[i]
                            csv_row[f"cabel_{i}_x"] = wire.x.item()
                            csv_row[f"cabel_{i}_y"] = wire.y.item()
                            csv_row[f"cabel_{i}_w"] = wire.w.item()
                            csv_row[f"cabel_{i}_h"] = wire.h.item()
                            print(f"Cable {i} location: {wire}")
                        else:
                            csv_row[f"cabel_{i}_x"] = 0
                            csv_row[f"cabel_{i}_y"] = 0
                            csv_row[f"cabel_{i}_w"] = 0
                            csv_row[f"cabel_{i}_h"] = 0

                    csv_writer.writerow(csv_row)

                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        raise NoMoreImages

                run_playback(img_src, callback)
finally:
    csv_file.close()
    cv2.destroyAllWindows()


state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
state of pad 1: unsoldered
state of pad 0: unsoldered
s

KeyboardInterrupt: 